In [1]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
import matplotlib.patches as patches
from matplotlib import pyplot as plt

from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

from extra_files import helper

%matplotlib inline

Using TensorFlow backend.
/home/aldo/Documents/ssd/data_generator/object_detection_2d_data_generator.py:43: UserWarning: 'BeautifulSoup' module is missing. The XML-parser will be unavailable.
  warnings.warn("'BeautifulSoup' module is missing. The XML-parser will be unavailable.")


In [2]:
def load_model_ssd300(path_model, 
                      size_model, 
                      num_classes, 
                      confidence, 
                      scales,
                      swap_channels,
                      mean_color,
                      divide_by_stddev,
                      steps,
                      iou,
                      alpha, 
                      expansion):
    '''
    Load model
    path_model: the path to the weights' model
    size_model: input size to the model
    num_classes: number of classes
    confidence: confidence use for nms
    scales: scales of default bounding boxes
    swap_channels: swap channels from input image
    mean_color: mean to rest from input image
    divide_by_stddev: number to divide input image
    steps: separations from each default box
    iou: iou use for nms
    alpha: controls the width of the network
    expansion: expansion on the inverted residual block from conv 13 on mobilenetv2
    '''
    
    # 1: Build the Keras model
    K.clear_session() # Clear previous models from memory.
    
    model = ssd_300(image_size=(size_model, size_model, 3),
                    n_classes=num_classes,
                    alpha = alpha,
                    expansion= expansion,
                    mode='inference',
                    scales=scales,
                    aspect_ratios_per_layer=[[1.0, 0.5, 2.0/3.0],
                                             [1.0, 0.5, 2.0/3.0, 1.0/3.0, 3.0/4.0],
                                             [1.0, 2.0, 0.5, 2.0/3.0, 1.0/3.0],
                                             [1.0, 2.0, 0.5, 2.0/3.0, 1.0/3.0],
                                             [1.0, 2.0, 0.5],
                                             [1.0, 2.0, 0.5]],
                    two_boxes_for_ar1=True,
                    steps=steps,
                    offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                    clip_boxes=False,
                    variances=[0.1, 0.1, 0.2, 0.2],
                    normalize_coords=True,
                    subtract_mean=mean_color,
                    divide_by_stddev=divide_by_stddev,
                    swap_channels=swap_channels,
                    confidence_thresh=confidence,
                    iou_threshold=iou,
                    top_k=200,
                    nms_max_output_size=400)

    # 2: Load the trained weights into the model.
    # TODO: Set the path of the trained weights.
    model.load_weights(path_model, by_name=True)

    # 3: Compile the model so that Keras won't complain the next time you load it.

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

    model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
    
    return model

In [3]:
def get_predictions(path_model, 
                    batch_size, 
                    images_list, 
                    size_model, 
                    num_classes,
                    scales,
                    swap_channels,
                    mean_color,
                    id_class, 
                    confidence, 
                    divide_by_stddev,
                    steps,
                    iou,
                    alpha,
                    expansion):
    '''
    Make predictions from a model
    path_model: the path to the weights' model
    batch_size: size of the batch
    images_list: list of images to make predictions
    size_model: input size to the model
    num_classes: number of classes
    scales: scales of default bounding boxes
    swap_channels: swap channels from input image
    mean_color: mean to rest from input image
    id_class: class id
    confidence: confidence use for nms
    divide_by_stddev: number to divide input image
    steps: separations from each default box
    iou: iou use for nms
    alpha: controls the width of the network
    expansion: expansion on the inverted residual block from conv 13 on mobilenetv2
    '''
    # Create variable to store predictions
    predictions = np.zeros(shape=(1, 200, 6))
    
    model = load_model_ssd300(path_model, 
                              size_model, 
                              num_classes, 
                              confidence, 
                              scales,
                              swap_channels,
                              mean_color,
                              divide_by_stddev,
                              steps,
                              iou,
                              alpha,
                              expansion)
    print(model.count_params())

    for batch in helper.get_batch(batch_size, images_list):
        pred = model.predict(batch)
        predictions = np.append(predictions, pred, axis=0)
    predictions = predictions[1:] # delete empty item

    clean_pre = helper.clean_predictions(predictions, id_class)
    adjust_pre = helper.adjust_predictions(clean_pre)
    K.clear_session()
    print('finished', path_model)
    return adjust_pre

In [4]:
## Load images
path_root = '../data-cic/preprocess_data'
test_images_320 = np.load(path_root + '/images_test_320x320.npy')
label_test_320 = np.load(path_root + '/label_test_320.npy')

In [5]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

962789
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_1.h5
Alpha 0.35 expansion 1
Number of images: 45
Presicion: 0.7006
Recall: 0.5693
F1 score: 0.6282


In [6]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

962789
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_1.h5
Alpha 0.35 expansion 1
Number of images: 45
Presicion: 0.6966
Recall: 0.5692
F1 score: 0.6265


In [7]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

997869
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_2.h5
Alpha 0.35 expansion 2
Number of images: 45
Presicion: 0.7351
Recall: 0.5158
F1 score: 0.6062


In [8]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

997869
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_2.h5
Alpha 0.35 expansion 2
Number of images: 45
Presicion: 0.733
Recall: 0.5155
F1 score: 0.6053


In [11]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1068029
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_4.h5
Alpha 0.35 expansion 4
Number of images: 45
Presicion: 0.7268
Recall: 0.5548
F1 score: 0.6292


In [5]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1068029
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_4.h5
Alpha 0.35 expansion 4
Number of images: 45
Presicion: 0.7219
Recall: 0.56
F1 score: 0.6308


In [6]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1138189
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.35_exp_6.h5
Alpha 0.35 expansion 6
Number of images: 45
Presicion: 0.6648
Recall: 0.5986
F1 score: 0.63


In [7]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.35
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.35 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1138189
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.35_exp_6.h5
Alpha 0.35 expansion 6
Number of images: 45
Presicion: 0.6641
Recall: 0.5612
F1 score: 0.6084


In [8]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1268836
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_1.h5
Alpha 0.5 expansion 1
Number of images: 45
Presicion: 0.7459
Recall: 0.5644
F1 score: 0.6426


In [9]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1268836
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_1.h5
Alpha 0.5 expansion 1
Number of images: 45
Presicion: 0.6661
Recall: 0.5402
F1 score: 0.5966


In [10]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1335044
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_2.h5
Alpha 0.5 expansion 2
Number of images: 45
Presicion: 0.7791
Recall: 0.5613
F1 score: 0.6525


In [11]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1335044
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_2.h5
Alpha 0.5 expansion 2
Number of images: 45
Presicion: 0.7785
Recall: 0.5495
F1 score: 0.6443


In [14]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1467460
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_4.h5
Alpha 0.5 expansion 4
Number of images: 45
Presicion: 0.739
Recall: 0.5762
F1 score: 0.6475


In [15]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1467460
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_4.h5
Alpha 0.5 expansion 4
Number of images: 45
Presicion: 0.739
Recall: 0.5762
F1 score: 0.6475


In [5]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1599876
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.5_exp_6.h5
Alpha 0.5 expansion 6
Number of images: 45
Presicion: 0.6931
Recall: 0.6393
F1 score: 0.6651


In [6]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.5
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.5 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1599876
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.5_exp_6.h5
Alpha 0.5 expansion 6
Number of images: 45
Presicion: 0.6939
Recall: 0.6279
F1 score: 0.6592


In [7]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1887396
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_1.h5
Alpha 0.75 expansion 1
Number of images: 45
Presicion: 0.7228
Recall: 0.6586
F1 score: 0.6892


In [8]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

1887396
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_1.h5
Alpha 0.75 expansion 1
Number of images: 45
Presicion: 0.7278
Recall: 0.6545
F1 score: 0.6892


In [9]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2024916
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_2.h5
Alpha 0.75 expansion 2
Number of images: 45
Presicion: 0.7516
Recall: 0.6453
F1 score: 0.6944


In [10]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2024916
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_2.h5
Alpha 0.75 expansion 2
Number of images: 45
Presicion: 0.752
Recall: 0.6569
F1 score: 0.7012


In [11]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2299956
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_4.h5
Alpha 0.75 expansion 4
Number of images: 45
Presicion: 0.7108
Recall: 0.6887
F1 score: 0.6996


In [12]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2299956
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_4.h5
Alpha 0.75 expansion 4
Number of images: 45
Presicion: 0.715
Recall: 0.6896
F1 score: 0.7021


In [13]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2574996
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_0.75_exp_6.h5
Alpha 0.75 expansion 6
Number of images: 45
Presicion: 0.7686
Recall: 0.6581
F1 score: 0.7091


In [14]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 0.75
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 0.75 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2574996
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_0.75_exp_6.h5
Alpha 0.75 expansion 6
Number of images: 45
Presicion: 0.7557
Recall: 0.656
F1 score: 0.7023


In [15]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2618612
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_1.h5
Alpha 1.0 expansion 1
Number of images: 45
Presicion: 0.7452
Recall: 0.6786
F1 score: 0.7104


In [16]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_1.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 1

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 1')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2618612
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_1.h5
Alpha 1.0 expansion 1
Number of images: 45
Presicion: 0.7215
Recall: 0.681
F1 score: 0.7007


In [17]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2852916
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_2.h5
Alpha 1.0 expansion 2
Number of images: 45
Presicion: 0.7892
Recall: 0.6269
F1 score: 0.6987


In [5]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_2.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 2

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 2')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

2852916
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_2.h5
Alpha 1.0 expansion 2
Number of images: 45
Presicion: 0.7873
Recall: 0.6216
F1 score: 0.6947


In [6]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

3321524
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_4.h5
Alpha 1.0 expansion 4
Number of images: 45
Presicion: 0.7859
Recall: 0.6298
F1 score: 0.6992


In [7]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_4.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 4

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 4')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

3321524
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_4.h5
Alpha 1.0 expansion 4
Number of images: 45
Presicion: 0.7859
Recall: 0.6333
F1 score: 0.7014


In [8]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

3790132
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_1.0_exp_6.h5
Alpha 1.0 expansion 6
Number of images: 45
Presicion: 0.7748
Recall: 0.6319
F1 score: 0.6961


In [9]:
# Make predictions of the models
from models.keras_ssdlite320_mobilenetv2 import ssd_300

path_model = '../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_6.h5'
batch_size = 32
images_list = test_images_320
size_model = 320
num_classes = 1
scales = [0.15, 0.33, 0.47, 0.61, 0.76, 0.90, 1.05]
swap_channels = False
mean_color = [-1., -1., -1.]
id_class = 1
confidence = 0.20
divide_by_stddev = [127.5, 127.5, 127.5]
steps = [16, 32, 64, 107, 160, 320]
iou = 0.45
alpha = 1.0
expansion = 6

predictions = get_predictions( path_model,
                               batch_size,
                               images_list,
                               size_model,
                               num_classes,
                               scales,
                               swap_channels,
                               mean_color,
                               id_class,
                               confidence,
                               divide_by_stddev,
                               steps,
                               iou,
                               alpha,
                               expansion)

print('Alpha 1.0 expansion 6')
predictions = helper.get_coordinates(predictions)
presicion, recall, f1_score = helper.cal_performance(label_test_320, predictions)

3790132
finished ../weights/models/tradeoff/ssdlite320_mobilenetv2_pascal_cic_f1_1.0_exp_6.h5
Alpha 1.0 expansion 6
Number of images: 45
Presicion: 0.7748
Recall: 0.6319
F1 score: 0.6961
